In [1]:
# import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tkinter import Tk, filedialog
import os
import time
import re
import requests
import subprocess
from datetime import date
import openpyxl
from selenium import webdriver                                     # automate web browser interaction
from selenium.webdriver.common.keys import Keys                    # automate keyboard actions
from selenium.webdriver.common.by import By                        # find elements by html id on webpage
from selenium.webdriver.support.ui import Select                   # automate dropdown selection
from selenium.webdriver.support.ui import WebDriverWait            # command driver to wait
from selenium.webdriver.support import expected_conditions as EC   # command driver to wait until loaded
from selenium.webdriver.chrome.options import Options              # option to make webdriver not visible
from selenium.common.exceptions import WebDriverException          # Deal with exptions in webdriver


In [2]:
ref = pd.read_excel("glansis_references_2024-05-05.xlsx", dtype = str)
print(ref.shape)
ref.head()


(25511, 8)


,species.id,scientific.name,common.name,status,group,refnum,type,reference
0,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,40167,Report,2015. Final Report: South Carolina State Wildl...
1,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41080,Journal Article,"Abdelmonem, A.A., M.M. Metwally, H.S. Hussein,..."
2,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,23905,Journal Article,"Aieta, A.E., and K. Oliveira. 2009. Distributi..."
3,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41061,Journal Article,"Ashworth, S.T., and C.R. Kennedy. 1999. Densit..."
4,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41062,Journal Article,"Ashworth, S.T., C.R. Kennedy, and G. Blanc. 19..."


In [3]:
# select only journal article references
ref = ref[ref['type'] == 'Journal Article']
print(ref['type'].unique())
print(ref.shape)
ref.head()


['Journal Article']
(13167, 8)


,species.id,scientific.name,common.name,status,group,refnum,type,reference
1,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41080,Journal Article,"Abdelmonem, A.A., M.M. Metwally, H.S. Hussein,..."
2,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,23905,Journal Article,"Aieta, A.E., and K. Oliveira. 2009. Distributi..."
3,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41061,Journal Article,"Ashworth, S.T., and C.R. Kennedy. 1999. Densit..."
4,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41062,Journal Article,"Ashworth, S.T., C.R. Kennedy, and G. Blanc. 19..."
5,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,40957,Journal Article,"Audenaert, V., T. Huyse, G. Goemans, C. Belpai..."


In [4]:
ref.drop_duplicates(subset=['refnum'], keep='first', inplace=True)
print(ref.shape)
ref.head()


(10185, 8)


,species.id,scientific.name,common.name,status,group,refnum,type,reference
1,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41080,Journal Article,"Abdelmonem, A.A., M.M. Metwally, H.S. Hussein,..."
2,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,23905,Journal Article,"Aieta, A.E., and K. Oliveira. 2009. Distributi..."
3,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41061,Journal Article,"Ashworth, S.T., and C.R. Kennedy. 1999. Densit..."
4,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41062,Journal Article,"Ashworth, S.T., C.R. Kennedy, and G. Blanc. 19..."
5,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,40957,Journal Article,"Audenaert, V., T. Huyse, G. Goemans, C. Belpai..."


In [5]:
# start web driver
driver = webdriver.Chrome()
driver.get('https://nas.er.usgs.gov/Signin.aspx')


In [15]:
# extract headers first
time_start = time.time()

# create empty list
file_names = []
href_links = []
ref_ids = []

# function to extract table from HTML
def find_table():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    file_table = soup.find('table', id = 'ContentPlaceHolder1_Table1')
    if file_table:
        for row in file_table.find_all('tr')[1:]:
            column = row.find("td")
            if column: 
                file_name = column.get_text(strip=True)
                href_link = column.find("a")["href"] if column.find("a") else None
                ref_id = soup.find('span', id = 'ContentPlaceHolder1_refnum').text
                
                # Append the text string and href link to their respective lists
                file_names.append(file_name)
                href_links.append(href_link)
                ref_ids.append(ref_id)
    else:
        file_name = 'NA'
        href_link = 'NA'
        ref_id = soup.find('span', id = 'ContentPlaceHolder1_refnum').text

        # Append the text string and href link to their respective lists
        file_names.append(file_name)
        href_links.append(href_link)
        ref_ids.append(ref_id)
        
                
    
    return file_table

# scrape references
for refnum in ref['refnum'].values:
    url = "https://nas.er.usgs.gov/DataEntry/References/ReferenceViewer.aspx?refnum="  + str(refnum)
    driver.get(url)
    find_table()

data = {"file.names": file_names, "href.links": href_links, "ref.id": ref_ids}
df = pd.DataFrame(data)

print(f'Full Reference Pull:{round((time.time()- time_start)/60)}')
print(round((time.time()- time_start)/60))

Full Reference Pull:28
28


In [16]:
df

,file.names,href.links,ref.id
0,41080.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,41080
1,23905.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,23905
2,41061.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,41061
3,41062.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,41062
4,41062_update.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,41062
...,...,...,...
9475,37577.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,37577
9476,37583.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,37583
9477,37586.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,37586
9478,36417.pdf,https://nas.er.usgs.gov\XUPLOADEDDOCSX/REFEREN...,36417


In [18]:
df.to_excel('pdf_linkss_' + str(date.today()) + '.xlsx', index = False)